## Clip score eval

In [1]:
from torchmetrics.functional.multimodal import clip_score
from functools import partial
import torch
### dont move to another cell
clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")

def calculate_clip_score(images, prompts):
    images_int = (images * 255).astype("uint8")
    clip_score = clip_score_fn(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    return round(float(clip_score), 4)

In [2]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceM4/COCO", split='test[0:100]')

tokens = dataset['sentences'][:100]

prompts = []
for token in tokens:
   prompts.append(token['raw'])

/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for HuggingFaceM4/COCO contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HuggingFaceM4/COCO
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
import diffusers
diffusers.logging.set_verbosity_warning()

diffusers.logging.disable_progress_bar()
import datasets
datasets.disable_progress_bar()

# Compare Different Models

In [14]:
from time import time
prompt = prompts[1]

In [8]:
print(prompt)

Man riding a motor bike on a dirt road on the countryside.


### LDM

In [28]:
from diffusers import DiffusionPipeline
model_id = "CompVis/ldm-text2im-large-256"

# load model and scheduler
ldm = DiffusionPipeline.from_pretrained(model_id)

cur = time()
ldm_image = ldm([prompt], num_inference_steps=75, eta=0.3, guidance_scale=8).images
print("Generating time in seconds: ", time()-cur)

vqvae/diffusion_pytorch_model.safetensors not found
The config attributes {'timestep_values': None} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


  0%|          | 0/75 [00:00<?, ?it/s]

Generating time in seconds:  371.7733271121979


### LCM

In [20]:
from diffusers import UNet2DConditionModel, DiffusionPipeline, LCMScheduler
import torch

unet = UNet2DConditionModel.from_pretrained("latent-consistency/lcm-sdxl", torch_dtype=torch.float16, variant="fp16")
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", unet=unet, torch_dtype=torch.float16, variant="fp16")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")
cur = time()
lcm_image = pipe(prompt=prompt, num_inference_steps=45, guidance_scale=8.0 ).images
print("Generating time in seconds: ", time()-cur)

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


  0%|          | 0/45 [00:00<?, ?it/s]

Generating time in seconds:  7.78581166267395


### LCM-LoRA

In [38]:
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
adapter_id = "latent-consistency/lcm-lora-sdxl"

sdxlpipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
sdxlpipe.scheduler = LCMScheduler.from_config(sdxlpipe.scheduler.config)
sdxlpipe.to("cuda")

# load and fuse lcm lora
sdxlpipe.load_lora_weights(adapter_id)
sdxlpipe.fuse_lora()
cur = time()
lora_image = sdxlpipe(prompt=prompt, num_inference_steps=45, guidance_scale=8.0).images
print("Generating time in seconds: ", time()-cur)

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


  0%|          | 0/45 [00:00<?, ?it/s]

Generating time in seconds:  13.801863431930542


### LDM-DPO

In [16]:
import torch
from diffusers import AutoPipelineForText2Image, DPMSolverMultistepScheduler
from diffusers.utils import make_image_grid

sdxl_dpo_lora_pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)

sdxl_dpo_lora_pipe.safety_checker = None
sdxl_dpo_lora_pipe.set_progress_bar_config(disable=True)


sdxl_dpo_lora_pipe.scheduler = DPMSolverMultistepScheduler.from_config(
    sdxl_dpo_lora_pipe.scheduler.config,
    use_karras_sigmas=True,
    algorithm_type="sde-dpmsolver++"
)

sdxl_dpo_lora_pipe.to("cuda");

seed = 12341234123
negative_prompt = "3d render, cartoon, drawing, art, low light, blur, pixelated, low resolution, black and white"
num_inference_steps = 40 # Keep
height = 1024
width = height

sdxl_dpo_lora_pipe.unload_lora_weights()
sdxl_dpo_lora_pipe.load_lora_weights(
    "radames/sdxl-DPO-LoRA",
    adapter_name="sdxl-dpo-lora",
)
# sdxl_dpo_lora_pipe.set_adapters(["sdxl-dpo-lora"], adapter_weights=[0.9])
generator = torch.Generator().manual_seed(seed)
cur = time()
dpo_image = sdxl_dpo_lora_pipe(disable_tqdm = True,prompt=prompt,num_images_per_prompt=1, num_inference_steps=45, guidance_scale=8.0).images
print("Generating time in seconds: ", time()-cur)

You are using `unload_lora_weights` to disable and unload lora weights. If you want to iteratively enable and disable adapter weights,you can use `pipe.enable_lora()` or `pipe.disable_lora()`. After installing the latest version of PEFT.


Generating time in seconds:  15.30357027053833


### Evaluation

In [29]:
# works with (, output_type='np')
import numpy as np
v14Image = ldm_image
clip_score = calculate_clip_score(np.array(v14Image), prompt)
print("CLIP Score: ", clip_score)

CLIP Score:  25.3381


In [30]:
from PIL import Image
import io
import os
import json

from warnings import filterwarnings


# os.environ["CUDA_VISIBLE_DEVICES"] = "0"    # choose GPU if you are on a multi GPU server
import numpy as np
import torch
import pytorch_lightning as pl
import torch.nn as nn
from torchvision import datasets, transforms
import tqdm

from os.path import join
from datasets import load_dataset
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import json

import clip


from PIL import Image, ImageFile


#####  This script will predict the aesthetic score for this image file:

img_path = "test.jpg"





# if you changed the MLP architecture during training, change it also here:
class MLP(pl.LightningModule):
    def __init__(self, input_size, xcol='emb', ycol='avg_rating'):
        super().__init__()
        self.input_size = input_size
        self.xcol = xcol
        self.ycol = ycol
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 128),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            #nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(64, 16),
            #nn.ReLU(),

            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
            x = batch[self.xcol]
            y = batch[self.ycol].reshape(-1, 1)
            x_hat = self.layers(x)
            loss = F.mse_loss(x_hat, y)
            return loss

    def validation_step(self, batch, batch_idx):
        x = batch[self.xcol]
        y = batch[self.ycol].reshape(-1, 1)
        x_hat = self.layers(x)
        loss = F.mse_loss(x_hat, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

def normalized(a, axis=-1, order=2):
    import numpy as np  # pylint: disable=import-outside-toplevel

    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)


In [31]:
aestheticPredictorModel = MLP(768)  # CLIP embedding dim is 768 for CLIP ViT L 14

s = torch.load("sac+logos+ava1-l14-linearMSE.pth")   # load the model you trained previously or the model available in this repo

aestheticPredictorModel.load_state_dict(s)

aestheticPredictorModel.to("cuda")
aestheticPredictorModel.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model2, preprocess = clip.load("ViT-L/14", device=device)  #RN50x64

In [32]:
# works with output type not 'np'
import ImageReward as reward
import clip
from PIL import Image
imageReward = reward.load("ImageReward-v1.0")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(v14Image[0]).unsqueeze(0).to(device)
text = clip.tokenize(prompt).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    hps = image_features @ text_features.T
    print("HPS: ",hps)

    v14_image_reward = imageReward.score(prompt, v14Image)
    image_features = model2.encode_image(image)

print("image reward: ", v14_image_reward)
im_emb_arr = normalized(image_features.cpu().detach().numpy() )
v14_aesthetic_score = aestheticPredictorModel(torch.from_numpy(im_emb_arr).to(device).type(torch.cuda.FloatTensor))
print("aesthetic score: ", v14_aesthetic_score)

load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
HPS:  tensor([[0.3140]], device='cuda:0', dtype=torch.float16)
image reward:  1.1446330547332764
aesthetic score:  tensor([[5.2901]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [33]:
v14Image[0].save("ldm_75.png")

## Latent Diffusion Model

https://huggingface.co/CompVis/ldm-text2im-large-256

In [4]:
from diffusers import DiffusionPipeline
model_id = "CompVis/ldm-text2im-large-256"

# load model and scheduler
ldm = DiffusionPipeline.from_pretrained(model_id)

# # run pipeline in inference (sample random noise and denoise)
# prompt = "A woman wearing a net on her head cutting a cake"
# images_ldm = ldm([prompt], num_inference_steps=50, eta=0.3, guidance_scale=6).images

# # save images
# for idx, image in enumerate(images_ldm):
#     image.save(f"ldm-{idx}.png")


/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
bert/model.safetensors not found
The config attributes {'timestep_values': None} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


In [5]:
total_clip_score = 0
for prompt in prompts:
  num_inference_steps = 75

  # image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np' ).images
  image = ldm([prompt], num_inference_steps=num_inference_steps, eta=0.3, guidance_scale=6, output_type='np').images

  img_clip_score = calculate_clip_score(image, prompt)
  total_clip_score += img_clip_score
  print(img_clip_score)

# clip score:
total_clip_score = total_clip_score/100
print(total_clip_score)

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Consistency Model

https://huggingface.co/dg845/consistency-model-pipelines

In [4]:
import torch

from diffusers import ConsistencyModelPipeline

device = "cuda"
# Load the cd_imagenet64_l2 checkpoint.
model_id_or_path = "dg845/consistency-model-pipelines"
pipe = ConsistencyModelPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe.to(device)

# Onestep Sampling
image = pipe(num_inference_steps=1).images[0]

/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
total_clip_score = 0
for prompt in prompts:
  num_inference_steps = 45

  image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np' ).images
  # image = ldm([prompt], num_inference_steps=num_inference_steps, eta=0.3, guidance_scale=6, output_type='np').images

  img_clip_score = calculate_clip_score(image, prompt)
  total_clip_score += img_clip_score
  print(img_clip_score)

# clip score:
total_clip_score = total_clip_score/100
print(total_clip_score)

TypeError: __call__() got an unexpected keyword argument 'prompt'

In [ ]:
import torch
import clip
from PIL import Image

num_inference_steps = 45
total_hps = 0

for i in range(len(prompts)):
    prompt = prompts[i]
    
    v14Image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    
    image = preprocess(v14Image).unsqueeze(0).to(device)
    text = clip.tokenize(prompt).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
    
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
        hps = image_features @ text_features.T
        print(hps)
    
    total_hps += hps

print(total_hps/100)

## Latent Consistency Model

In [ ]:
# !git clone https://github.com/huggingface/transformers.git
# %cd transformers
# !pip install -e .

https://huggingface.co/latent-consistency/lcm-sdxl

In [4]:
from diffusers import UNet2DConditionModel, DiffusionPipeline, LCMScheduler
import torch

unet = UNet2DConditionModel.from_pretrained("latent-consistency/lcm-sdxl", torch_dtype=torch.float16, variant="fp16")
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", unet=unet, torch_dtype=torch.float16, variant="fp16")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# prompt = "a close-up picture of an old man standing in the rain"

# image = pipe(prompt, num_inference_steps=4, guidance_scale=8.0).images[0]

/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.26.3",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "LCMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [6]:
total_clip_score = 0
for prompt in prompts:
  num_inference_steps = 45

  image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np' ).images
  # image = ldm([prompt], num_inference_steps=num_inference_steps, eta=0.3, guidance_scale=6, output_type='np').images

  img_clip_score = calculate_clip_score(image, prompt)
  total_clip_score += img_clip_score
  print(img_clip_score)

# clip score:
total_clip_score = total_clip_score/100
print(total_clip_score)

  0%|          | 0/45 [00:00<?, ?it/s]

31.9215


  0%|          | 0/45 [00:00<?, ?it/s]

33.19


  0%|          | 0/45 [00:00<?, ?it/s]

...

33.5582


  0%|          | 0/45 [00:00<?, ?it/s]

27.1192


  0%|          | 0/45 [00:00<?, ?it/s]

29.7487
30.797236999999985


In [63]:
import torch
import clip
from PIL import Image

num_inference_steps = 45
total_hps = 0

for i in range(len(prompts)):
    prompt = prompts[i]
    
    v14Image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    
    image = preprocess(v14Image).unsqueeze(0).to(device)
    text = clip.tokenize(prompt).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
    
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
        hps = image_features @ text_features.T
        print(hps)
    
    total_hps += hps

print(total_hps/100)

  0%|          | 0/45 [00:00<?, ?it/s]

tensor([[0.3633]], device='cuda:0', dtype=torch.float16)


  0%|          | 0/45 [00:00<?, ?it/s]

tensor([[0.3296]], device='cuda:0', dtype=torch.float16)


  0%|          | 0/45 [00:00<?, ?it/s]

...

tensor([[0.2627]], device='cuda:0', dtype=torch.float16)


  0%|          | 0/45 [00:00<?, ?it/s]

tensor([[0.2871]], device='cuda:0', dtype=torch.float16)


  0%|          | 0/45 [00:00<?, ?it/s]

tensor([[0.2927]], device='cuda:0', dtype=torch.float16)
tensor([[0.3096]], device='cuda:0', dtype=torch.float16)


In [7]:

num_inference_steps = 45
total_image_reward = 0
total_aesthetic_score = 0


import torch
import ImageReward as reward

pipe.safety_checker = None

for i in range(len(prompts)):
    prompt = prompts[i]
    
    
    v14Image = pipe(disable_tqdm = True,prompt=prompt,num_images_per_prompt=1, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
    pipe.set_progress_bar_config(disable=True)
    
    image = preprocess(v14Image).unsqueeze(0).to(device)
    imageReward = reward.load("ImageReward-v1.0")
    with torch.no_grad():
        v14_image_reward = imageReward.score(prompt, v14Image)
        image_features = model2.encode_image(image)
    
    total_image_reward += v14_image_reward
    print("image reward: ", v14_image_reward)
    im_emb_arr = normalized(image_features.cpu().detach().numpy() )
    v14_aesthetic_score = aestheticPredictorModel(torch.from_numpy(im_emb_arr).to(device).type(torch.cuda.FloatTensor))
    total_aesthetic_score += v14_aesthetic_score
    print("aesthetic score: ", v14_aesthetic_score)


print("==================total=================")
print("average image reward: ")
print(total_image_reward/100)
print("average aesthetic score: ")
print(total_aesthetic_score/100)



  0%|          | 0/45 [00:00<?, ?it/s]

load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  1.286228895187378
aesthetic score:  tensor([[5.8616]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  1.5407110452651978
aesthetic score:  tensor([[5.6564]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  1.1631336212158203
aesthetic score:  tensor([[5.4862]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  0.2987986207008362
aesthetic score:  tensor([[6.2472]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  1.787033200263977
aesthetic score:  tensor([[

## LCM - LORA

https://huggingface.co/latent-consistency/lcm-lora-sdv1-5

In [6]:
# https://huggingface.co/latent-consistency/lcm-lora-sdxl

import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
adapter_id = "latent-consistency/lcm-lora-sdxl"

sdxlpipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
sdxlpipe.scheduler = LCMScheduler.from_config(sdxlpipe.scheduler.config)
sdxlpipe.to("cuda")

# load and fuse lcm lora
sdxlpipe.load_lora_weights(adapter_id)
sdxlpipe.fuse_lora()

/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/research/jenzheng/.conda/envs/cs229/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configurat

In [5]:
import torch
import clip
from PIL import Image

num_inference_steps = 45
total_hps = 0

for i in range(len(prompts)):
    prompt = prompts[i]
    
    v14Image = sdxlpipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    
    image = preprocess(v14Image).unsqueeze(0).to(device)
    text = clip.tokenize(prompt).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
    
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
        hps = image_features @ text_features.T
        print(hps)
    
    total_hps += hps

print(total_hps/100)

  0%|          | 0/45 [00:00<?, ?it/s]

tensor([[0.2822]], device='cuda:0', dtype=torch.float16)


  0%|          | 0/45 [00:00<?, ?it/s]

tensor([[0.2300]], device='cuda:0', dtype=torch.float16)


  0%|          | 0/45 [00:00<?, ?it/s]

...

tensor([[0.2113]], device='cuda:0', dtype=torch.float16)


  0%|          | 0/45 [00:00<?, ?it/s]

tensor([[0.2374]], device='cuda:0', dtype=torch.float16)


  0%|          | 0/45 [00:00<?, ?it/s]

tensor([[0.2473]], device='cuda:0', dtype=torch.float16)
tensor([[0.2742]], device='cuda:0', dtype=torch.float16)


In [8]:

num_inference_steps = 45
total_image_reward = 0
total_aesthetic_score = 0


import torch
import ImageReward as reward

sdxlpipe.safety_checker = None

for i in range(len(prompts)):
    prompt = prompts[i]
    
    
    v14Image = sdxlpipe(disable_tqdm = True,prompt=prompt,num_images_per_prompt=1, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
    sdxlpipe.set_progress_bar_config(disable=True)
    
    image = preprocess(v14Image).unsqueeze(0).to(device)
    imageReward = reward.load("ImageReward-v1.0")
    with torch.no_grad():
        v14_image_reward = imageReward.score(prompt, v14Image)
        image_features = model2.encode_image(image)
    
    total_image_reward += v14_image_reward
    print("image reward: ", v14_image_reward)
    im_emb_arr = normalized(image_features.cpu().detach().numpy() )
    v14_aesthetic_score = aestheticPredictorModel(torch.from_numpy(im_emb_arr).to(device).type(torch.cuda.FloatTensor))
    total_aesthetic_score += v14_aesthetic_score
    print("aesthetic score: ", v14_aesthetic_score)


print("==================total=================")
print("average image reward: ")
print(total_image_reward/100)
print("average aesthetic score: ")
print(total_aesthetic_score/100)



  0%|          | 0/45 [00:00<?, ?it/s]

load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  -1.1388907432556152
aesthetic score:  tensor([[5.1207]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  0.2732652425765991
aesthetic score:  tensor([[5.3916]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  0.21898427605628967
aesthetic score:  tensor([[5.0814]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  -1.1769771575927734
aesthetic score:  tensor([[5.4639]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  0.8105146884918213
aesthetic score:  tens

## Image Reward

In [8]:
!pip install image-reward

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 5.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 12.4 MB/s eta 0:00:0000:01
  Created wheel for fairscale: filename=fairscale-0.4.13-py3-none-any.whl size=332108 sha256=02c3c9ac3d600f5291fda21ec08f9bb1c546d53bd075dfaf34d9ae74910460db
  Stored in directory: /home/research/jenzheng/.cache/pip/wheels/10/ea/7f/8f35af83599829bb4790bdc16949dd99aeeb62e9a1faf47d47
Successfully built fairscale


In [10]:

num_inference_steps = 1
# higher inf step == higher clip score?


v14Image_total_image_reward = 0

import os
import torch
import ImageReward as reward


for i in range(len(prompts)):
  prompt = prompts[i]


  v14Image = pipe(disable_tqdm = True,prompt=prompt,num_images_per_prompt=1, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
  sd_pipeline.set_progress_bar_config(disable=True)

  imageReward = reward.load("ImageReward-v1.0")
  with torch.no_grad():
    v14_image_reward = imageReward.score(prompt, v14Image)

  v14Image_total_image_reward += v14_image_reward
  if i % 100 == 0:
    print("-------iteration ", i)
    print('image reward for V1-4:', v14_image_reward)
    print('avg image reward for V1-4:', v14Image_total_image_reward/(i+1))

NameError: name 'pipe' is not defined

## Aesthetic Predictor

In [16]:
!pip install pytorch_lightning

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 10.6 MB/s eta 0:00:0000:0100:01


In [4]:
from PIL import Image
import io
import os
import json

from warnings import filterwarnings


# os.environ["CUDA_VISIBLE_DEVICES"] = "0"    # choose GPU if you are on a multi GPU server
import numpy as np
import torch
import pytorch_lightning as pl
import torch.nn as nn
from torchvision import datasets, transforms
import tqdm

from os.path import join
from datasets import load_dataset
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import json

import clip


from PIL import Image, ImageFile


#####  This script will predict the aesthetic score for this image file:

img_path = "test.jpg"





# if you changed the MLP architecture during training, change it also here:
class MLP(pl.LightningModule):
    def __init__(self, input_size, xcol='emb', ycol='avg_rating'):
        super().__init__()
        self.input_size = input_size
        self.xcol = xcol
        self.ycol = ycol
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 128),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            #nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(64, 16),
            #nn.ReLU(),

            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
            x = batch[self.xcol]
            y = batch[self.ycol].reshape(-1, 1)
            x_hat = self.layers(x)
            loss = F.mse_loss(x_hat, y)
            return loss

    def validation_step(self, batch, batch_idx):
        x = batch[self.xcol]
        y = batch[self.ycol].reshape(-1, 1)
        x_hat = self.layers(x)
        loss = F.mse_loss(x_hat, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

def normalized(a, axis=-1, order=2):
    import numpy as np  # pylint: disable=import-outside-toplevel

    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)


In [5]:
aestheticPredictorModel = MLP(768)  # CLIP embedding dim is 768 for CLIP ViT L 14

s = torch.load("sac+logos+ava1-l14-linearMSE.pth")   # load the model you trained previously or the model available in this repo

aestheticPredictorModel.load_state_dict(s)

aestheticPredictorModel.to("cuda")
aestheticPredictorModel.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model2, preprocess = clip.load("ViT-L/14", device=device)  #RN50x64

In [ ]:

sd_pipeline.safety_checker = None

num_inference_steps = 70
# higher inf step == higher clip score?


v14Image_total_aesthetic_score = 0

import os
import torch



for i in range(len(prompts)):
  prompt = prompts[i]

  v14Image = sd_pipeline(disable_tqdm = True,prompt=prompt,num_images_per_prompt=1, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
  sd_pipeline.set_progress_bar_config(disable=True)
  image = preprocess(v14Image).unsqueeze(0).to(device)

  with torch.no_grad():
    image_features = model2.encode_image(image)


  im_emb_arr = normalized(image_features.cpu().detach().numpy() )

  v14_aesthetic_score = aestheticPredictorModel(torch.from_numpy(im_emb_arr).to(device).type(torch.cuda.FloatTensor))


  v14Image_total_aesthetic_score += v14_aesthetic_score
  if i % 100 == 0:
    print("-------iteration ", i)
    print('aesthetic score for V1-4:', v14_aesthetic_score)
    print('avg aesthetic score for V1-4:', v14Image_total_aesthetic_score/(i+1))


## Human Preference

In [6]:
# !pip install ftfy regex tqdm
# !pip install git+https://github.com/openai/CLIP.git

  Using cached ftfy-6.1.3-py3-none-any.whl.metadata (6.2 kB)
Using cached ftfy-6.1.3-py3-none-any.whl (53 kB)
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-uz37rfiw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-uz37rfiw
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Using cached torchvision-0.17.1-cp39-cp39-manylinux1_x86_64.whl.metadata (6.6 kB)
Using cached torchvision-0.17.1-cp39-cp39-manylinux1_x86_64.whl (6.9 MB)
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=db61f746ba5dae8eae2c998ebc528a714a4ebad2fae7a06a0998849c735c30cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-5yliroo1/wheels/c8/e4/e1/11374c111387672fc2068dfbe0d4b424cb9cdd1b2e184a71b5
Successfully built clip


In [65]:
import torch
import clip
from PIL import Image

num_inference_steps = 45
total_hps = 0

for i in range(len(prompts)):
    prompt = prompts[i]
    
    v14Image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    
    image = preprocess(v14Image).unsqueeze(0).to(device)
    text = clip.tokenize(prompt).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
    
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
        hps = image_features @ text_features.T
        print(hps)
    
    total_hps += hps

print(total_hps/100)

## Clipscore evaluation

In [ ]:
total_clip_score = 0
for prompt in prompts:
  num_inference_steps = 75

  image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np' ).images

  img_clip_score = calculate_clip_score(image, prompt)
  total_clip_score += img_clip_score
  print(img_clip_score)

# clip score:
total_clip_score = total_clip_score/100
print(total_clip_score)

## DPO

In [9]:
import torch
from diffusers import AutoPipelineForText2Image, DPMSolverMultistepScheduler
from diffusers.utils import make_image_grid

sdxl_dpo_lora_pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)

sdxl_dpo_lora_pipe.safety_checker = None
sdxl_dpo_lora_pipe.set_progress_bar_config(disable=True)


sdxl_dpo_lora_pipe.scheduler = DPMSolverMultistepScheduler.from_config(
    sdxl_dpo_lora_pipe.scheduler.config,
    use_karras_sigmas=True,
    algorithm_type="sde-dpmsolver++"
)

sdxl_dpo_lora_pipe.to("cuda");

seed = 12341234123
prompt = "professional portrait photo of a girl, photograph, highly detailed face, depth of field, moody light, golden hour, style by Dan Winters, Russell James, Steve McCurry, centered, extremely detailed, Nikon D850, award winning photography"
negative_prompt = "3d render, cartoon, drawing, art, low light, blur, pixelated, low resolution, black and white"
num_inference_steps = 40 # Keep
height = 1024
width = height
guidance_scale = 7.5

sdxl_dpo_lora_pipe.unload_lora_weights()
sdxl_dpo_lora_pipe.load_lora_weights(
    "radames/sdxl-DPO-LoRA",
    adapter_name="sdxl-dpo-lora",
)
# sdxl_dpo_lora_pipe.set_adapters(["sdxl-dpo-lora"], adapter_weights=[0.9])
generator = torch.Generator().manual_seed(seed)

You are using `unload_lora_weights` to disable and unload lora weights. If you want to iteratively enable and disable adapter weights,you can use `pipe.enable_lora()` or `pipe.disable_lora()`. After installing the latest version of PEFT.


In [5]:
total_clip_score = 0
for prompt in prompts:
  num_inference_steps = 75

  image = sdxl_dpo_lora_pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np' ).images

  img_clip_score = calculate_clip_score(image, prompt)
  total_clip_score += img_clip_score
  print(img_clip_score)

# clip score:
total_clip_score = total_clip_score/100
print(total_clip_score)

Unused or unrecognized kwargs: padding.


32.5879


Unused or unrecognized kwargs: padding.


31.784


Unused or unrecognized kwargs: padding.


...

30.185


Unused or unrecognized kwargs: padding.


31.4924


Unused or unrecognized kwargs: padding.


26.5593
31.74810599999998


In [ ]:
total_clip_score

In [6]:
import torch
import clip
from PIL import Image

num_inference_steps = 75
total_hps = 0

for i in range(len(prompts)):
    prompt = prompts[i]
    
    v14Image = sdxl_dpo_lora_pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    
    image = preprocess(v14Image).unsqueeze(0).to(device)
    text = clip.tokenize(prompt).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
    
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
        hps = image_features @ text_features.T
        print(hps)
    
    total_hps += hps

print(total_hps/100)

tensor([[0.3726]], device='cuda:0', dtype=torch.float16)
tensor([[0.3303]], device='cuda:0', dtype=torch.float16)
tensor([[0.2935]], device='cuda:0', dtype=torch.float16)
tensor([[0.3633]], device='cuda:0', dtype=torch.float16)
tensor([[0.3550]], device='cuda:0', dtype=torch.float16)
tensor([[0.3364]], device='cuda:0', dtype=torch.float16)
tensor([[0.3403]], device='cuda:0', dtype=torch.float16)
tensor([[0.3345]], device='cuda:0', dtype=torch.float16)
tensor([[0.3462]], device='cuda:0', dtype=torch.float16)
tensor([[0.3086]], device='cuda:0', dtype=torch.float16)
tensor([[0.3486]], device='cuda:0', dtype=torch.float16)
tensor([[0.3364]], device='cuda:0', dtype=torch.float16)
tensor([[0.3374]], device='cuda:0', dtype=torch.float16)
tensor([[0.3047]], device='cuda:0', dtype=torch.float16)
tensor([[0.3477]], device='cuda:0', dtype=torch.float16)
tensor([[0.2510]], device='cuda:0', dtype=torch.float16)
tensor([[0.3022]], device='cuda:0', dtype=torch.float16)
tensor([[0.3059]], device='cuda

In [10]:

num_inference_steps = 75
total_image_reward = 0
total_aesthetic_score = 0


import torch
import ImageReward as reward

sdxl_dpo_lora_pipe.safety_checker = None

for i in range(len(prompts)):
    prompt = prompts[i]
    
    
    v14Image = sdxl_dpo_lora_pipe(disable_tqdm = True,prompt=prompt,num_images_per_prompt=1, num_inference_steps=num_inference_steps, guidance_scale=8.0).images[0]
    sdxl_dpo_lora_pipe.set_progress_bar_config(disable=True)
    
    image = preprocess(v14Image).unsqueeze(0).to(device)
    imageReward = reward.load("ImageReward-v1.0")
    with torch.no_grad():
        v14_image_reward = imageReward.score(prompt, v14Image)
        image_features = model2.encode_image(image)
    
    total_image_reward += v14_image_reward
    print("image reward: ", v14_image_reward)
    im_emb_arr = normalized(image_features.cpu().detach().numpy() )
    v14_aesthetic_score = aestheticPredictorModel(torch.from_numpy(im_emb_arr).to(device).type(torch.cuda.FloatTensor))
    total_aesthetic_score += v14_aesthetic_score
    print("aesthetic score: ", v14_aesthetic_score)


print("==================total=================")
print("average image reward: ")
print(total_image_reward/100)
print("average aesthetic score: ")
print(total_aesthetic_score/100)



load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  1.7381075620651245
aesthetic score:  tensor([[5.7677]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  1.7845016717910767
aesthetic score:  tensor([[6.2547]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  0.3458864390850067
aesthetic score:  tensor([[5.4840]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  1.4757494926452637
aesthetic score:  tensor([[6.1185]], device='cuda:0', grad_fn=<AddmmBackward0>)
load checkpoint from /home/research/jenzheng/.cache/ImageReward/ImageReward.pt
checkpoint loaded
image reward:  1.876319169998169
aesthetic score:  tensor([